第十四章  可迭代的对象、迭代器和生成器

扫描内存中放不下的数据集时，我们要找到一种惰性获取数据项的方式，即按需一次获取一个数据项。这就是迭代器模式。

所有生成器都是迭代器，因为生成器完全实现了迭代器的接口。在python社区中，大多数时候把迭代器和生成器视作一个概念。

14.1 Sentence类第1版：单词序列

In [45]:
#senten.py 把句子划分为单词序列
import re
import reprlib

RE_WORD=re.compile('\w+')

class Sentence:
    def __init__(self,text):
        self.text=text
        self.words=RE_WORD.findall(text)
    
    def __getitem__(self, position):
        return self.words[position]
    
    def __len__(self):
        return len(self.words)
    
    def __repr__(self):
        return 'Sentence(%s)'%reprlib.repr(self.text)

In [22]:
s=Sentence('"The time has come," the Walrus said')
s

Sentence('"The time ha...e Walrus said')

In [23]:
iter(s)

In [24]:
s.__dict__

{'text': '"The time has come," the Walrus said',
 'words': ['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']}

In [5]:
for word in s:#Sentence实例可以迭代！
    print(word)

The
time
has
come
the
Walrus
said


In [6]:
list(s)

['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']

In [7]:
s[0]

'The'

In [8]:
s[-1]

'said'

![chapter14-2](image/chapter14-2.jpg)

14.2 可迭代对象与迭代器的对比

可迭代对象：使用iter内置函数可以获取迭代器的对象。如果对象实现了能返回迭代器的__iter__方法，那么对象是可迭代的。序列都可以迭代；实现了__getitem__方法，而且其参数是从0开始的索引，这种对象也可迭代。

明确可迭代对象和迭代器之间的关系：python从可迭代对象中获取迭代器。

In [9]:
s='ABC'
for char in s:
    print(char)

A
B
C


In [10]:
s='ABC'
it=iter(s)#使用可迭代对象构建迭代器it
while True:
    try:print(next(it))#不断在迭代器上调用next函数，获取下一个字符
    except StopIteration:#没字符了，抛出异常
        del it#释放对it的引用，即废弃迭代器对象
        break

A
B
C


标准的迭代器接口有两个方法：

__next__

__iter__返回self，实例本身，以便在需要使用可迭代对象的地方使用迭代器，例如for循环

![chapter14-4](image/chapter14-4.jpg)

In [11]:
s3=Sentence('Pig and Pepper')
it=iter(s3)
it

In [12]:
next(it)

'Pig'

In [13]:
next(it)

'and'

In [14]:
next(it)

'Pepper'

In [15]:
next(it) #迭代器没有单词了，抛出StopIteration异常

StopIteration: 

In [17]:
list(it) #到头后，迭代器没用了

[]

In [18]:
list(iter(s3)) #想再次迭代，需重构迭代器

['Pig', 'and', 'Pepper']

14.3 Sentence类第2版：典型的迭代器

In [25]:
import re
import reprlib

RE_WORD=re.compile('\w+')

class Sentence:
    def __init__(self,text):
        self.text=text
        self.words=RE_WORD.findall(text)

    def __repr__(self):
        return 'Sentence(%s)'%reprlib.repr(self.text)
    
    def __iter__(self):#与前一版相比，这里只多了一个__iter__方法。这里没__getitem方法，为的是明确表明这个类可以迭代是因为实现了__iter__方法
        return SentenceIterator(self.words) #返回一个SentenceIterator实例

#构建SentenceIterator
class SentenceIterator:
    def __init__(self,words):
        self.words=words
        self.index=0
    
    def __next__(self):
        try:
            word=self.words[self.index]
        except IndexError:
            raise StopIteration()
        self.index+=1
        return word
    
    def __iter__(self):
        return self

In [26]:
s=Sentence('"The time has come," the Walrus said')
s

Sentence('"The time ha...e Walrus said')

In [27]:
for char in s:
    print(char)

The
time
has
come
the
Walrus
said


In [29]:
s.__dict__

{'text': '"The time has come," the Walrus said',
 'words': ['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']}

In [31]:
iter(s)
    

In [34]:
it=iter(s)
it2=iter(it)
it,it2

(<__main__.SentenceIterator at 0x1bcaf356160>,
 <__main__.SentenceIterator at 0x1bcaf356160>)

把Sentence变成迭代器：坏主意

可迭代对象有个__iter__方法，每次都可实例化一个新的迭代器；而迭代器要实现__next__方法，返回单个元素；此外还要实现__iter__方法，返回迭代器本身。

除了__iter__方法除外，你可能还想再Sentence类实现__next__,让Sentence实例既是可迭代对象又是自身的迭代器，但这种想法十分糟糕。

注意，迭代器模式可用来：

(1).访问一个聚合对象的内容而无需暴露它的内部表示

(2).支持对聚合对象多种遍历

(3).为遍历不同的聚合结构提供一个统一的接口（即支持多态迭代）

为了“支持多种遍历”，必须能从同一个可迭代的实例中获取多个独立的迭代器，而且各个迭代器要能维护自身的内部状态，因此这一模式的正确实现方式是，每次调用iter(my iterable)新建一个独立的迭代器。这就是为什么这个示例需要定义SentenceIterator类。


可迭代对象一定不能是自身的迭代器。也就是说，可迭代对象必须实现__iter__方法，但不能实现__next__方法。
另一方面，迭代器应该一直可以迭代。迭代器的__iter__方法应该返回自身。


关于迭代器为什么一定要有__iter__方法，简单的说，如果迭代器不实现_iter_方法的话，那么迭代器本身就不能迭代了，不是可迭代对象。

比如下面这个例子，自己定义的SentenceIterator如果不实现_iter_方法的话，就不可迭代了，违反了迭代器一定是可迭代对象的约定

In [49]:
import re
import reprlib

RE_WORD=re.compile('\w+')

class Sentence:
    def __init__(self,text):
        self.text=text
        self.words=RE_WORD.findall(text)

    def __repr__(self):
        return 'Sentence(%s)'%reprlib.repr(self.text)
    
    def __iter__(self):#与前一版相比，这里只多了一个__iter__方法。这里没__getitem方法，为的是明确表明这个类可以迭代是因为实现了__iter__方法
        return SentenceIterator(self.words) #返回一个SentenceIterator实例

#构建SentenceIterator
class SentenceIterator:
    def __init__(self,words):
        self.words=words
        self.index=0
    
    def __next__(self):
        try:
            word=self.words[self.index]
        except IndexError:
            raise StopIteration()
        self.index+=1
        return word
    

In [50]:
s=Sentence('"The time has come," the Walrus said')
s

Sentence('"The time ha...e Walrus said')

In [51]:
iter(s)

In [52]:
it=iter(s)
for i in it:
    print(i)

TypeError: 'SentenceIterator' object is not iterable

14.4 Sentence类第三版：生成器函数

实现相同功能，但却符合python习惯的方式是，用生成器函数代替SentenceIterator类。

In [53]:
import re
import reprlib

RE_WORD=re.compile('\w+')

class Sentence:
    def __init__(self,text):
        self.text=text
        self.words=RE_WORD.findall(text)

    def __repr__(self):
        return 'Sentence(%s)'%reprlib.repr(self.text)
    
    #这里的__iter__方法是生成器函数
    def __iter__(self):
        for word in self.words:#迭代self.words
            yield word#产出当前word
        return #return这个语句不是必要的，这个函数可以直接落空，自动返回。不管有没有return语句，
               # 生成器函数都不会抛出StopIteration，而是在生成完全部值后直接退出

In [54]:
s=Sentence('"The time has come," the Walrus said')
s

Sentence('"The time ha...e Walrus said')

In [55]:
for i in s:
    print(i)

The
time
has
come
the
Walrus
said


In [57]:
it=iter(s)
it

<generator object __iter__ at 0x000001BCAF280FC0>

In [58]:
for i in it:
    print(i)

The
time
has
come
the
Walrus
said


只要python函数中包含关键字yield，该函数就是生成器函数。调用生成器函数返回一个生成器对象。

生成器函数的定义体中一般有循环，但这不是必要条件如下例：

In [59]:
def gen_123():
    yield 1
    yield 2
    yield 3

In [60]:
gen_123

<function __main__.gen_123>

In [61]:
gen_123()

<generator object gen_123 at 0x000001BCAF342B48>

In [62]:
for i in gen_123():#生成器是迭代器，会生成传给yield关键字的表达式的值
    print(i)

1
2
3


In [68]:
g=gen_123()#g是迭代器
next(g)

1

In [64]:
next(g)

2

In [65]:
next(g)

3

In [66]:
next(g)

StopIteration: 

In [70]:
g=gen_123()#g是迭代器
next(g)
for i in g:
    print(i)

2
3


把生成器传给next(...)时，生成器函数会向前，执行函数定义体下一个yield语句，返回产出的值，并在函数定义体的当前位置暂停。最终，函数的定义体返回时，外层的生成器对象会抛出StopIteration异常——这一点与迭代器协议一致。

In [71]:
def gen_AB():
    print('start')
    yield  'A'
    print('continue')
    yield 'B'
    print('end')
    yield 'C'
for c in gen_AB():#迭代时，for机制的作用与g=iter(gen_AB())一样，用于获取生成器对象，然后每次迭代时隐式调用next(g)
    print('-->',c)

start
--> A
continue
--> B
end
--> C


In [72]:
next(gen_AB())

start


'A'

14.5 Sentence类第四版：惰性实现

目前实现的几版Sentence类都不具有惰性，因为__init__方法急迫的构建了文本中的单词列表，然后将其绑定到self.words上，并不节省内存。如果只需迭代前几个单词，大多数工作是白费力气。

re.finditer函数是re.findall函数的惰性版本，返回的不是一个列表，而是一个生成器，按需生产re.MatchObject实例。节省内存

In [73]:
import re
import reprlib

RE_WORD = re.compile('\w+')


class Sentence:

    def __init__(self, text):
        self.text = text  #不再需words列表

    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

    def __iter__(self):
        for match in RE_WORD.finditer(self.text): #finditer函数构建一个迭代器，包含self.texxt中匹配RE_WORD的单词，产出MatchObject对象
            yield match.group()

In [74]:
s=Sentence('"The time has come," the Walrus said')
s

Sentence('"The time ha...e Walrus said')

In [82]:
for i in s:#迭代时，for机制的作用与g=iter(s)一样，用于获取生成器对象，然后每次迭代时隐式调用next(g)
    print(i)

The
time
has
come
the
Walrus
said


In [83]:
it=iter(s)
it

<generator object __iter__ at 0x000001BCAF342E08>

In [84]:
next(it)

'The'

14.6 Sentence第五版：生成器表达式

简单的生成器函数，可以替换为生成器表达式

生成器表达式可以理解为列表推导的惰性版本：不会迫切的构建列表，而是返回一个生成器，按需惰性生成元素。

In [85]:
def gen_AB():
    print('start')
    yield  'A'
    print('continue')
    yield 'B'
    print('end')
    yield 'C'

In [86]:
res1=[x*3 for x in gen_AB()]#列表推导迫切的迭代gen_AB()函数生成的生成器对象产出的元素：A,B,C。注意，下面的输出。

start
continue
end


In [87]:
for i in res1:#这个循环迭代列表推导生成的res1列表
    print('-->',i)

--> AAA
--> BBB
--> CCC


In [89]:
res2=(x*3 for x in gen_AB())#一个生成器对象

In [90]:
res2

<generator object <genexpr> at 0x000001BCAF360410>

In [91]:
for i in res2:
    print('-->',i)

start
--> AAA
continue
--> BBB
end
--> CCC


In [94]:
import re
import reprlib

RE_WORD = re.compile('\w+')


class Sentence:

    def __init__(self, text):
        self.text = text  #不再需words列表

    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

    def __iter__(self):
        return (match.group() for match in RE_WORD.finditer(self.text))

In [95]:
s=Sentence('"The time has come," the Walrus said')
s
for i in s:
    print(i)

The
time
has
come
the
Walrus
said
